Importing Libraries

In [ ]:
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience

Connecting to Neo4j

In [ ]:
uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))
session = driver.session()

gds = GraphDataScience(uri, auth=("neo4j", "password"))

Ingesting Data to Neo4j

In [ ]:
# Ingest data into neo4j 
gds.run_cypher('''CALL apoc.periodic.iterate(
  "LOAD CSV WITH HEADERS FROM 'file:///fraudTestSample.csv' AS row RETURN row",
  "CREATE (p:Person {first: row.first, last: row.last, gender: row.gender, job: row.job})
  CREATE (m:Merchant {merchantName: row.merchant})
  CREATE (c:Category {categoryType: row.category})
  CREATE (txn:Transaction {txnID: row.trans_num, amount: toFloat(row.amt), timestamp: row.trans_date_trans_time, isFraud: toInteger(row.txn_is_fraud)})
  CREATE (cc:CreditCard {ccNum: row.cc_num})
  
  MERGE (zip:Zipcode {code: row.zip})
  MERGE (city:City {name: row.city})
  ON CREATE SET city.population = toInteger(row.city_pop)
  MERGE (state:State {name: row.state})
  
  MERGE (merch_zip:Zipcode {code: row.merch_zip_code})
  MERGE (merch_city:City {name: row.merch_po_name})
  MERGE (merch_state:State {name: row.merch_state})

  CREATE (p)-[:OWNS]->(cc)
  CREATE (cc)-[:MADE_A]->(txn)
  CREATE (txn)-[:PAID]->(m)
  CREATE (m)-[:IS_A]->(c)
  CREATE (p)-[:LOCATED_IN]->(zip)
  CREATE (zip)-[:LOCATED_IN]->(city)
  CREATE (city)-[:LOCATED_IN]->(state)
  CREATE (m)-[:LOCATED_IN]->(merch_zip)
  CREATE (merch_zip)-[:LOCATED_IN]->(merch_city)
  CREATE (merch_city)-[:LOCATED_IN]->(merch_state)",
  {batchSize: 500, parallel: false}
)''')

Stats of data

In [ ]:
# Show stats about the ingested data
gds.run_cypher("""
CALL apoc.meta.stats()
    YIELD labels
    UNWIND keys(labels) AS nodeLabel
    RETURN nodeLabel, labels[nodeLabel] AS nodeCount
""")

In [ ]:
# total relationship counts
gds.run_cypher('''
    CALL apoc.meta.stats()
    YIELD relTypesCount
    UNWIND keys(relTypesCount) AS relationshipType
    RETURN relationshipType, relTypesCount[relationshipType] AS relationshipCount
''')

Show_result function

In [ ]:
def show_result(query):
    result = session.run(query)
    print(result.data())

Queries

No of is_fraud counts

In [ ]:
show_result('''
MATCH (t:Transaction)
RETURN t.isFraud, count(*) AS Count
ORDER BY t.isFraud;
''')

Fraud Transaction Paths
-Trace the complete path of fraudulent transactions through the graph

In [ ]:
show_result('''
MATCH paths = (p:Person)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(t:Transaction)-[:PAID]->(m:Merchant)-[:IS_A]->(c:Category)
WHERE t.isFraud = 1
RETURN paths
LIMIT 10;
''')

Merchant Fraud Networks
-Find merchants that share fraudulent transactions with the same credit cards

In [ ]:
show_result('''
MATCH (cc:CreditCard)-[:MADE_A]->(t1:Transaction)-[:PAID]->(m1:Merchant),
      (cc)-[:MADE_A]->(t2:Transaction)-[:PAID]->(m2:Merchant)
WHERE t1.isFraud = 1 AND t2.isFraud = 1 AND m1 <> m2
RETURN m1.merchantName AS Merchant1, m2.merchantName AS Merchant2, COUNT(DISTINCT cc) AS SharedFraudulentCards
ORDER BY SharedFraudulentCards DESC;
''')

Geographic Fraud Clusters
-Identify geographic clusters of fraud by traversing location hierarchies

In [ ]:
show_result('''
MATCH (t:Transaction)-[:PAID]->(m:Merchant)-[:LOCATED_IN]->(:Zipcode)-[:LOCATED_IN]->(c:City)-[:LOCATED_IN]->(s:State)
WHERE t.isFraud = 1
WITH s, c, COUNT(t) AS fraudCount
ORDER BY fraudCount DESC
RETURN s.name AS state, COLLECT({city: c.name, count: fraudCount}) AS fraudCities
LIMIT 10;
''')

Credit Card Multi-Hop Fraud Risk
-Find credit cards at high risk based on connection to fraudulent merchant

In [ ]:
show_result('''
MATCH (cc:CreditCard)-[:MADE_A]->(t1:Transaction)-[:PAID]->(m:Merchant)
WHERE t1.isFraud = 0
WITH cc, m, COUNT(t1) AS legitimateTransactions
MATCH (otherCC:CreditCard)-[:MADE_A]->(t2:Transaction)-[:PAID]->(m)
WHERE t2.isFraud = 1 AND otherCC <> cc
WITH cc, COUNT(DISTINCT otherCC) AS riskConnections, SUM(legitimateTransactions) AS totalLegitTxns
WHERE riskConnections > 2
MATCH (p:Person)-[:OWNS]->(cc)
RETURN p.first + ' ' + p.last AS cardHolder, cc.ccNum AS creditCard, 
       riskConnections, totalLegitTxns,
       (1.0 * riskConnections / totalLegitTxns) AS riskScore
ORDER BY riskScore DESC;
''')

Circular Fraud Patterns
-Detect circular patterns where the same person uses multiple cards with the same merchants

In [ ]:
show_result('''
MATCH cycle = (p:Person)-[:OWNS]->(cc1:CreditCard)-[:MADE_A]->(t1:Transaction)-[:PAID]->(m:Merchant)<-[:PAID]-(t2:Transaction)<-[:MADE_A]-(cc2:CreditCard)<-[:OWNS]-(p)
WHERE cc1 <> cc2 AND (t1.isFraud = 1 OR t2.isFraud = 1)
RETURN p.first + ' ' + p.last AS person, 
       COLLECT(DISTINCT cc1.ccNum) + COLLECT(DISTINCT cc2.ccNum) AS creditCards,
       COUNT(DISTINCT m) AS merchantCount,
       LENGTH(cycle) AS cycleLength
ORDER BY merchantCount DESC;
''')


Common Fraud Path Analysis
-Analyze most common paths leading to fraud

In [ ]:
show_result('''
MATCH path = (start)-[*1..4]->(t:Transaction)
WHERE t.isFraud = 1
WITH [node IN NODES(path) | LABELS(node)[0]] AS nodeTypes, COUNT(*) AS frequency
RETURN nodeTypes, frequency
ORDER BY frequency DESC
LIMIT 10;
''')

Shortest Path Between Fraudulent Transactions
-Find connections between seemingly unrelated fraud events

In [ ]:
show_result('''
MATCH (t1:Transaction), (t2:Transaction)
WHERE t1.isFraud = 1 AND t2.isFraud = 1 
AND t1.txnID < t2.txnID // Avoid duplicates
WITH t1, t2
MATCH path = shortestPath((t1)-[*..6]-(t2))
WHERE LENGTH(path) > 1
RETURN t1.txnID AS fraud1, t2.txnID AS fraud2, 
       [node IN NODES(path) | LABELS(node)[0] + ': ' + 
         CASE 
           WHEN 'txnID' IN KEYS(node) THEN node.txnID
           WHEN 'ccNum' IN KEYS(node) THEN node.ccNum
           WHEN 'merchantName' IN KEYS(node) THEN node.merchantName
           WHEN 'first' IN KEYS(node) THEN node.first + ' ' + node.last
           ELSE ''
         END
       ] AS connectionPath,
       LENGTH(path) AS pathLength
ORDER BY pathLength
LIMIT 20;
''')

Fraud Communities Detection
-Identify communities of connected entities involved in fraud

In [ ]:
show_result('''
MATCH (t:Transaction)
WHERE t.isFraud = 1
MATCH (t)-[*1..2]-(entity)
WITH COLLECT(DISTINCT entity) AS fraudConnectedEntities
UNWIND fraudConnectedEntities AS e1
UNWIND fraudConnectedEntities AS e2
WITH e1, e2
WHERE ID(e1) < ID(e2) // Ensure unique pairs
MATCH (e1)-[*1..3]-(commonEntity)-[*1..3]-(e2)
RETURN e1, e2, COUNT(DISTINCT commonEntity) AS commonConnections
ORDER BY commonConnections DESC
LIMIT 20;
''')

Centrality-Based Fraud Risk
-Use centrality measures to find key entities in fraud networks

In [ ]:
show_result('''
MATCH (t:Transaction)
WHERE t.isFraud = 1
MATCH (t)-[:PAID]->(m:Merchant)
WITH COLLECT(DISTINCT m) AS fraudMerchants
UNWIND fraudMerchants AS fraudMerchant
MATCH (fraudMerchant)<-[:PAID]-(t:Transaction)<-[:MADE_A]-(cc:CreditCard)
WITH fraudMerchant, cc, COUNT(t) AS transactionCount
ORDER BY transactionCount DESC
WITH fraudMerchant, COLLECT({cc: cc.ccNum, count: transactionCount})[0..5] AS topCards,
     SUM(transactionCount) AS totalTransactions
RETURN fraudMerchant.merchantName AS merchantName, 
       topCards AS topCreditCards,
       totalTransactions AS totalFraudTransactions,
       SIZE((fraudMerchant)<-[:PAID]-()) AS inDegree,
       SIZE((fraudMerchant)-[:IS_A]->()) AS outDegree
ORDER BY inDegree DESC;
''')

Categorical Fraud Co-occurrence
-Find categories that frequently co-occur in fraudulent transactions

In [ ]:
show_result('''
MATCH (c1:Category)<-[:IS_A]-(m1:Merchant)<-[:PAID]-(t1:Transaction)<-[:MADE_A]-(cc:CreditCard)-[:MADE_A]->(t2:Transaction)-[:PAID]->(m2:Merchant)-[:IS_A]->(c2:Category)
WHERE t1.isFraud = 1 AND t2.isFraud = 1 AND c1 <> c2
AND t1.txnID <> t2.txnID
RETURN c1.categoryType AS category1, 
       c2.categoryType AS category2, 
       COUNT(DISTINCT cc) AS sharedCards
ORDER BY sharedCards DESC
LIMIT 15;
''')

PageRank for Key Fraud Entities
-Demonstrates how to project a graph and run the PageRank algorithm to identify key entities in a fraud network

In [ ]:
show_result('''
CALL gds.graph.project(
  'fraudNetwork',
  ['Person', 'CreditCard', 'Transaction', 'Merchant', 'Category'],
  {
    OWNS: {orientation: 'UNDIRECTED'},
    MADE_A: {orientation: 'UNDIRECTED'},
    PAID: {orientation: 'UNDIRECTED'},
    IS_A: {orientation: 'UNDIRECTED'}
  },
  {
    nodeProperties: {
      isFraud: {
        property: 'isFraud',
        defaultValue: 0 // Ensure this is an integer, not a string
      }
    }
  }
);
''')

In [ ]:
show_result('''
CALL gds.pageRank.stream('fraudNetwork', {
  dampingFactor: 0.85,
  maxIterations: 20
})
YIELD nodeId, score
WITH gds.util.asNode(nodeId) AS node, score
WHERE (node:Person OR node:Merchant OR node:CreditCard)
RETURN 
    LABELS(node)[0] AS entityType,
    CASE
        WHEN node:Person THEN node.first + ' ' + node.last
        WHEN node:Merchant THEN node.merchantName
        WHEN node:CreditCard THEN node.ccNum
        ELSE ''
    END AS entityName,
    score AS centralityScore,
    SIZE([(node)-[r]-() | r]) AS connectionCount
ORDER BY centralityScore DESC
LIMIT 20;
''')

Node Similarity for Transaction Pattern Detection
-Create projection of transaction network with key properties

In [ ]:
show_result('''
CALL gds.graph.project(
  'txnSimilarity',
  ['Transaction', 'CreditCard', 'Merchant'],
  {
    MADE_A: {orientation: 'NATURAL'},
    PAID: {orientation: 'NATURAL'}
  },
  {
    nodeProperties: ['amount', 'isFraud']
  }
);


''')

In [ ]:
show_result('''
CALL gds.nodeSimilarity.stream('txnSimilarity')
YIELD node1, node2, similarity
WHERE similarity > 0.7
WITH gds.util.asNode(node1) AS txn1, gds.util.asNode(node2) AS txn2, similarity
WHERE txn1.txnID < txn2.txnID // Avoid duplicates
MATCH (txn1)<-[:MADE_A]-(cc1:CreditCard)
MATCH (txn2)<-[:MADE_A]-(cc2:CreditCard)
MATCH (txn1)-[:PAID]->(m1:Merchant)
MATCH (txn2)-[:PAID]->(m2:Merchant)
RETURN 
    txn1.txnID AS txn1ID,
    txn2.txnID AS txn2ID,
    similarity,
    txn1.isFraud AS txn1IsFraud,
    txn2.isFraud AS txn2IsFraud,
    cc1.ccNum = cc2.ccNum AS sameCard,
    m1.merchantName = m2.merchantName AS sameMerchant,
    ABS(txn1.amount - txn2.amount) AS amountDifference
ORDER BY similarity DESC
LIMIT 20;
''')

Triangle Counting for Fraud Network Analysis
-Demonstrates how to project a graph and use the Triangle Count algorithm to identify entities involved in triangular relationships within a fraud network. The query also calculates a fraudNetworkScore based on the number of triangles and connected fraudulent transactions.

In [ ]:
show_result('''
CALL gds.graph.project(
  'fraudTriangles',
  ['Person', 'CreditCard', 'Transaction', 'Merchant'],
  {
    OWNS: {orientation: 'UNDIRECTED'},
    MADE_A: {orientation: 'UNDIRECTED'},
    PAID: {orientation: 'UNDIRECTED'}
  },
  {
    nodeProperties: ['isFraud']
  }
);

''')

In [ ]:
show_result('''
CALL gds.triangleCount.stream('fraudTriangles')
YIELD nodeId, triangleCount
WITH gds.util.asNode(nodeId) AS node, triangleCount
WHERE triangleCount > 0
// Match transactions linked to the node that are marked as fraudulent
MATCH (node)-[*1..2]-(t:Transaction)
WHERE toInteger(t.isFraud) = 1
WITH node, triangleCount, COUNT(DISTINCT t) AS connectedFraudTxns
WHERE connectedFraudTxns > 0
RETURN 
    LABELS(node)[0] AS entityType,
    CASE
        WHEN node:Person THEN node.first + ' ' + node.last
        WHEN node:Merchant THEN node.merchantName
        WHEN node:CreditCard THEN node.ccNum
        WHEN node:Transaction THEN node.txnID
        ELSE 'Unknown'
    END AS entityName,
    triangleCount,
    connectedFraudTxns,
    (1.0 * triangleCount * connectedFraudTxns) AS fraudNetworkScore
ORDER BY fraudNetworkScore DESC
LIMIT 20;
''')

Query: Find transactions where the amount is unusually high compared to the average transaction amount for a given person.

In [ ]:
show_result('''
MATCH (p:Person)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)
WITH p, avg(txn.amount) AS avgAmount, stdev(txn.amount) AS stdDevAmount
MATCH (p)-[:OWNS]->(cc)-[:MADE_A]->(txn)
WHERE txn.amount > avgAmount + 2 * stdDevAmount
RETURN p.first, p.last, txn.amount, avgAmount, stdDevAmount, txn.isFraud AS isFraud
ORDER BY txn.amount DESC;
''')


Query: Find transactions where the person's location and the merchant's location are far apart (e.g., more than 1000 km).

In [ ]:
show_result('''
MATCH (p:Person)-[:LOCATED_AT]->(personLoc:Geocode),
      (m:Merchant)-[:LOCATED_AT]->(merchantLoc:Geocode),
      (p)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)-[:PAID]->(m)
WITH point.distance(point({latitude: personLoc.latitude, longitude: personLoc.longitude}),
                    point({latitude: merchantLoc.latitude, longitude: merchantLoc.longitude})) AS distance,
      p, m, txn
WHERE distance > 1000000  // Distance in meters (1000 km)
RETURN p.first, p.last, m.merchantName, txn.amount, distance
ORDER BY distance DESC
LIMIT 1000;
''')

Query: Analyze the time of day or day of the week when fraudulent transactions are most likely to occur.

In [ ]:
show_result('''
MATCH (txn:Transaction)
WHERE txn.isFraud = true
WITH txn, datetime(txn.timestamp) AS datetime
RETURN date(datetime) AS date, datetime.hour AS hour, count(*) AS fraudCount
ORDER BY fraudCount DESC;
''')

Query: Identify merchants with a high percentage of fraudulent transactions.

In [ ]:
show_result('''
MATCH (m:Merchant)<-[:PAID]-(txn:Transaction)
WITH m, count(txn) AS totalTransactions, 
     sum(CASE WHEN toInteger(txn.isFraud) = 1 THEN 1 ELSE 0 END) AS fraudTransactions
WHERE fraudTransactions > 0
RETURN m.merchantName, totalTransactions, fraudTransactions,
       toFloat(fraudTransactions) / totalTransactions AS fraudRate
ORDER BY fraudRate DESC;
''')

Query: Find credit cards that were used in multiple cities within a 24-hour period.

In [ ]:
show_result('''
MATCH (cc:CreditCard)-[:MADE_A]->(txn:Transaction)-[:PAID]->(m:Merchant)-[:LOCATED_AT]->(merchantLoc:Geocode)-[:IS_IN]->(merch_zip:Zipcode)-[:IS_IN]->(merch_city:City)
WHERE txn.amount > 1000  // Filter early to reduce data size
WITH cc, merch_city, apoc.date.parse(txn.timestamp, 's', 'MM-dd-yyyy HH:mm') AS txnTime
ORDER BY cc, txnTime
WITH cc, collect(merch_city.name) AS cities, collect(txnTime) AS times
WHERE size(cities) > 1 AND duration.between(datetime({epochSeconds: times[0]}), datetime({epochSeconds: times[1]})).hours < 24
RETURN cc.ccNum, count(cities)
ORDER BY cc.ccNum;
''')

Query: Identify transactions where the spending pattern deviates significantly from a person's historical behavior (e.g., sudden large purchases).

In [ ]:
show_result('''
MATCH (p:Person)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)
WITH p, avg(txn.amount) AS avgAmount, stdev(txn.amount) AS stdDevAmount
MATCH (p)-[:OWNS]->(cc)-[:MADE_A]->(txn)
WHERE txn.amount > avgAmount + 3 * stdDevAmount
RETURN p.first, p.last, txn.amount, avgAmount, stdDevAmount
ORDER BY txn.amount DESC;
''')

Query: Identify transactions that occur in rapid succession (e.g., multiple transactions within a few minutes).

In [ ]:
show_result('''
MATCH (cc:CreditCard)-[:MADE_A]->(txn1:Transaction),
      (cc)-[:MADE_A]->(txn2:Transaction)
WHERE txn1 <> txn2 AND 
      duration.between(
          datetime({epochSeconds: apoc.date.parse(txn1.timestamp, 's', 'dd-MM-yyyy HH:mm')}), 
          datetime({epochSeconds: apoc.date.parse(txn2.timestamp, 's', 'dd-MM-yyyy HH:mm')})
      ).minutes < 5
RETURN cc.ccNum, txn1.txnID AS txn1, txn2.txnID AS txn2, txn1.timestamp AS time1, txn2.timestamp AS time2
ORDER BY cc.ccNum;
''')

Query: Identify people who own multiple credit cards that have been used in fraudulent transactions.

In [ ]:
show_result('''
MATCH (p:Person)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)
WHERE txn.isFraud = true
WITH p, count(DISTINCT cc) AS fraudCardCount
WHERE fraudCardCount > 1
RETURN p.first, p.last, fraudCardCount
ORDER BY fraudCardCount DESC;
''')

Query: Find transactions where the merchant is located in a state that is unusual for the person (e.g., a person from California making frequent transactions in New York).

In [ ]:
show_result('''
MATCH (p:Person)-[:LOCATED_AT]->(personLoc:Geocode)-[:IS_IN]->(zip:Zipcode)-[:IS_IN]->(city:City)-[:IS_IN]->(state:State),
      (m:Merchant)-[:LOCATED_AT]->(merchantLoc:Geocode)-[:IS_IN]->(merch_zip:Zipcode)-[:IS_IN]->(merch_city:City)-[:IS_IN]->(merch_state:State),
      (p)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)-[:PAID]->(m)
WHERE state <> merch_state
WITH p, merch_state, count(*) AS outOfStateTransactions
WHERE outOfStateTransactions > 5
RETURN p.first, p.last, merch_state.name AS merchantState, outOfStateTransactions
ORDER BY outOfStateTransactions DESC;
''')

Query: Identify transactions where a person frequently transacts with a merchant that is unusual for their demographic (e.g., a young person frequently shopping at a luxury store).

In [ ]:
show_result('''
MATCH (p:Person)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)-[:PAID]->(m:Merchant)-[:IS_A]->(c:Category)
WHERE p.age < 30 AND c.categoryType = 'Luxury'
WITH p, m, count(*) AS luxuryTransactions
WHERE luxuryTransactions > 3
RETURN p.first, p.last, m.merchantName, luxuryTransactions
ORDER BY luxuryTransactions DESC;
''')

Query: Find transactions where the distance between the person's location and the merchant's location is unusually large.

In [ ]:
show_result('''
MATCH (p:Person)-[:LOCATED_AT]->(personLoc:Geocode),
      (m:Merchant)-[:LOCATED_AT]->(merchantLoc:Geocode),
      (p)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)-[:PAID]->(m)
WITH p, m, point.distance(point({latitude: personLoc.latitude, longitude: personLoc.longitude}),
                          point({latitude: merchantLoc.latitude, longitude: merchantLoc.longitude})) AS distance
WHERE distance > 500000  // 500 km
RETURN p.first, p.last, m.merchantName, distance
ORDER BY distance DESC;
''')

Query: Find credit cards that have been used in multiple locations within a short time frame.

In [ ]:
show_result('''
MATCH (cc:CreditCard)-[:MADE_A]->(txn:Transaction)-[:PAID]->(m:Merchant)-[:LOCATED_AT]->(merchantLoc:Geocode)
WHERE txn.amount > 100  // Filter early to reduce data size
WITH cc, txn, merchantLoc, 
     apoc.date.parse(txn.timestamp, 'ms', 'dd-MM-yyyy HH:mm') AS parsedTimestamp
WITH cc, txn, merchantLoc, 
     datetime({epochMillis: parsedTimestamp}) AS datetime
ORDER BY cc.ccNum, datetime
WITH cc, collect(merchantLoc) AS locations, collect(datetime) AS times
WHERE size(locations) > 1 AND duration.between(times[0], times[1]).minutes < 1
RETURN cc.ccNum
ORDER BY cc.ccNum
LIMIT 1000;
''')

Query: Identify transactions where the merchant category is unusual for the person (e.g., a person who typically shops at grocery stores suddenly makes a high-value transaction at a luxury store).

In [ ]:
show_result('''
MATCH (p:Person)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)-[:PAID]->(m:Merchant)-[:IS_A]->(c:Category)
WITH p, c, count(*) AS categoryCount, avg(txn.amount) AS avgAmount
MATCH (p)-[:OWNS]->(cc)-[:MADE_A]->(txn)-[:PAID]->(m)-[:IS_A]->(c)
WHERE txn.amount > 2 * avgAmount AND categoryCount < 5
RETURN p.first, p.last, c.categoryType, txn.amount, avgAmount
ORDER BY txn.amount DESC;
''')


Query: Find transactions where the person's location and the transaction timestamp suggest they couldn't have physically been at the merchant's location (e.g., two transactions in different cities within a short time frame).

In [ ]:
show_result('''
MATCH (p:Person)-[:LOCATED_AT]->(personLoc:Geocode),
      (m:Merchant)-[:LOCATED_AT]->(merchantLoc:Geocode),
      (p)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(txn:Transaction)-[:PAID]->(m)
WHERE txn.amount > 1000  // Filter early to reduce data size
WITH p, personLoc, merchantLoc, txn, 
     apoc.date.parse(txn.timestamp, 'ms', 'dd-MM-yyyy HH:mm') AS parsedTimestamp
WITH p, personLoc, merchantLoc, txn, 
     datetime({epochMillis: parsedTimestamp}) AS txnTime
ORDER BY p, txnTime
WITH p, collect(personLoc) AS personLocs, collect(merchantLoc) AS merchantLocs, collect(txnTime) AS txnTimes, count(txn) AS numTransactions, 
     sum(CASE WHEN txn.isFraud = 1 THEN 1 ELSE 0 END) AS numFraudTransactions
WHERE size(personLocs) > 1
AND numTransactions > 1
AND point.distance(point({latitude: personLocs[0].latitude, longitude: personLocs[0].longitude}),
                  point({latitude: merchantLocs[0].latitude, longitude: merchantLocs[0].longitude})) > 100000
AND duration.between(txnTimes[0], txnTimes[1]).minutes < 30
RETURN p.first, p.last, numTransactions, numFraudTransactions
ORDER BY numTransactions DESC
LIMIT 1000;
''')


Fraud Risk Scoring with PostgreSQL and Neo4j
-This combines transaction data from PostgreSQL and network analysis from Neo4j to compute a comprehensive fraud risk score for each customer.


In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Database connection parameters
pg_params = {
    'database': 'mydatabase',
    'user': 'myuser',
    'password': 'mypassword',
    'host': 'localhost',
    'port': 5432
}

neo4j_params = {
    'uri': 'neo4j://localhost:7687',
    "auth": ("neo4j", "password")
}

# Connect to databases
def connect_to_pg():
    return psycopg2.connect(**pg_params)

def connect_to_neo4j():
    return GraphDatabase.driver(**neo4j_params)

# ----------- HYBRID ANALYSIS EXAMPLES -----------

def hybrid_analysis_1_fraud_risk_scoring():
    """
    Combines PostgreSQL transaction metrics with Neo4j network analysis
    to create a comprehensive fraud risk score for each customer.
    """
    # Step 1: Get transaction patterns from PostgreSQL
    with connect_to_pg() as pg_conn:
        cursor = pg_conn.cursor()
        cursor.execute("""
            SELECT cc_num, first, last,
                   COUNT(trans_num) AS transaction_count,
                   AVG(amt) AS avg_amount,
                   MAX(amt) AS max_amount,
                   STDDEV(amt) AS stddev_amount
            FROM transactions 
            GROUP BY cc_num, first, last
        """)
        pg_results = cursor.fetchall()
        pg_df = pd.DataFrame(pg_results, columns=[
            'cc_num', 'first_name', 'last_name',
            'transaction_count', 'avg_amount', 'max_amount', 'stddev_amount'
        ])

    # Step 2: Get network metrics from Neo4j
    with connect_to_neo4j() as neo4j_driver:
        session = neo4j_driver.session()
        neo4j_results = session.run("""
            MATCH (p:Person)-[:OWNS]->(cc:CreditCard)-[:MADE_A]->(t:Transaction)
            OPTIONAL MATCH (t)-[:PAID]->(m:Merchant)
            WHERE t.isFraud = 1
            WITH p, count(m) AS fraud_merchant_count
            OPTIONAL MATCH (p)-[:OWNS]->(cc)-[:MADE_A]->(txn)-[:PAID]->(m)-[:LOCATED_IN]->(zip)
            WITH p, fraud_merchant_count, count(DISTINCT zip) AS connected_locations
            RETURN p.cc_num AS cc_num,
                   fraud_merchant_count,
                   connected_locations
        """)
        neo4j_df = pd.DataFrame([record.values() for record in neo4j_results],
                                columns=['cc_num', 'fraud_merchant_count', 'connected_locations'])

    # Step 3: Combine the results
    combined_df = pd.merge(pg_df, neo4j_df, on='cc_num')

    # Step 4: Calculate a combined risk score
    combined_df['amount_risk'] = (
        (combined_df['max_amount'] / combined_df['avg_amount']) *
        (combined_df['stddev_amount'] / combined_df['avg_amount'])
    )
    combined_df['network_risk'] = combined_df['fraud_merchant_count'] * (
        0.1 + combined_df['connected_locations'] / 100
    )
    combined_df['combined_risk_score'] = combined_df['amount_risk'] + combined_df['network_risk'] * 5

    # Return top 20 customers by risk score
    return combined_df.sort_values('combined_risk_score', ascending=False).head(20)

# ----------- VISUALIZATION -----------

def visualize_results(risk_scores):
    plt.figure(figsize=(12, 6))
    
    if isinstance(risk_scores, pd.DataFrame) and not risk_scores.empty:
        plt.subplot(1, 1, 1)
        sns.scatterplot(
            data=risk_scores,
            x='network_risk',
            y='amount_risk',
            size='combined_risk_score',
            hue='fraud_merchant_count',
            alpha=0.7,
            palette='coolwarm'
        )
        plt.title('Customer Fraud Risk Analysis')
        plt.xlabel('Network Risk')
        plt.ylabel('Amount Risk')
    
    plt.tight_layout()
    plt.savefig('fraud_risk_analysis.png')
    plt.show()


In [ ]:
print("Running Hybrid Analysis 1: Fraud Risk Scoring")
risk_scores = hybrid_analysis_1_fraud_risk_scoring()
print(f"Found {len(risk_scores)} customers with high risk scores")
    
# Visualize results
visualize_results(risk_scores)